In [71]:
import caffe

In [72]:
import numpy as np

In [73]:
model_1fcs = 'nets/tracker_1_fcs.prototxt';
model_2fcs = 'nets/tracker_2_fcs.prototxt';
model_3fcs = 'nets/tracker.prototxt';
model_ensemble = 'nets/tracker_3_2_1.prototxt';
weights_1fcs = 'dump_model_1_fcs_iter_50000.caffemodel';
weights_2fcs = 'dump_model_2_fcs_iter_50000.caffemodel';
weights_3fcs = 'dump_model_5_iter_50000.caffemodel';

solver = 'nets/solver.prototxt'

caffe.set_mode_gpu();
caffe.set_device(0);

In [74]:
net_1fcs = caffe.Net(model_1fcs, weights_1fcs, caffe.TEST)
net_2fcs = caffe.Net(model_2fcs, weights_2fcs, caffe.TEST)
net_3fcs = caffe.Net(model_3fcs, weights_3fcs, caffe.TEST)
net_ensemble = caffe.Net(model_ensemble, weights_3fcs, caffe.TEST)

#### Make sure that all conv weights are the same!

In [75]:
# Make sure that all weights in all models are the same
for param in range(1,6, 1):
    key = 'conv{}'.format(param)
    key_p = 'conv{}_p'.format(param)
    for k in [key, key_p]:
        conv_1fcs = net_1fcs.params[k]
        conv_2fcs = net_2fcs.params[k]
        conv_3fcs = net_3fcs.params[k]
        conv_ensemble = net_ensemble.params[k]
        
        assert(np.all(conv_1fcs[0].data == conv_2fcs[0].data))
        assert(np.all(conv_2fcs[0].data == conv_3fcs[0].data))
        assert(np.all(conv_3fcs[0].data == conv_ensemble[0].data))
        assert(np.all(conv_1fcs[1].data == conv_2fcs[1].data))
        assert(np.all(conv_2fcs[1].data == conv_3fcs[1].data))
        assert(np.all(conv_3fcs[1].data == conv_ensemble[1].data))
        print("All four networks have the same weights and biases for layer {}!".format(k))

All four networks have the same weights and biases for layer conv1!
All four networks have the same weights and biases for layer conv1_p!
All four networks have the same weights and biases for layer conv2!
All four networks have the same weights and biases for layer conv2_p!
All four networks have the same weights and biases for layer conv3!
All four networks have the same weights and biases for layer conv3_p!
All four networks have the same weights and biases for layer conv4!
All four networks have the same weights and biases for layer conv4_p!
All four networks have the same weights and biases for layer conv5!
All four networks have the same weights and biases for layer conv5_p!


#### Transfer the weights to new model

In [76]:
net_ensemble.params

OrderedDict([('conv1', <caffe._caffe.BlobVec at 0x7fd52d2e5210>),
             ('conv2', <caffe._caffe.BlobVec at 0x7fd52d2e53d0>),
             ('conv3', <caffe._caffe.BlobVec at 0x7fd52d2e5d70>),
             ('conv4', <caffe._caffe.BlobVec at 0x7fd52d2e5de0>),
             ('conv5', <caffe._caffe.BlobVec at 0x7fd52d2e5c20>),
             ('conv1_p', <caffe._caffe.BlobVec at 0x7fd52d2e5c90>),
             ('conv2_p', <caffe._caffe.BlobVec at 0x7fd52d2e5ec0>),
             ('conv3_p', <caffe._caffe.BlobVec at 0x7fd52d2e5f30>),
             ('conv4_p', <caffe._caffe.BlobVec at 0x7fd52d2e5fa0>),
             ('conv5_p', <caffe._caffe.BlobVec at 0x7fd52d2e5d00>),
             ('1fcs-fc7-newb_new', <caffe._caffe.BlobVec at 0x7fd52d2e5e50>),
             ('1fcs-fc8-shapes-new', <caffe._caffe.BlobVec at 0x7fd52d2d1050>),
             ('2fcs-fc7-new_new', <caffe._caffe.BlobVec at 0x7fd52d2d10c0>),
             ('2fcs-fc7-newb_new', <caffe._caffe.BlobVec at 0x7fd52d2d1130>),
             ('2f

In [77]:
def transfer_params(from_net, from_layer, to_net, to_layer):
    '''
    Moves both weights and biases between two networks
    '''
    assert from_net.params[from_layer][0].data.shape == to_net.params[to_layer][0].data.shape, "Weights should be of the same weights!"
    assert from_net.params[from_layer][1].data.shape == to_net.params[to_layer][1].data.shape, "Biases should be of the same weights!"
    
    to_net.params[to_layer][0].data[...] = from_net.params[from_layer][0].data[...]
    to_net.params[to_layer][1].data[...] = from_net.params[from_layer][1].data[...]
    
    # Make sure changes took effect!
    assert np.all(from_net.params[from_layer][0].data == to_net.params[to_layer][0].data)
    assert np.all(from_net.params[from_layer][1].data == to_net.params[to_layer][1].data)

In [78]:
# 1FCS
for layer in ['fc7-newb_new', 'fc8-shapes-new']:
    transfer_params(
        net_1fcs, layer,
        net_ensemble, '1fcs-{}'.format(layer)
    )

In [79]:
# 2FCS
for layer in ["fc7-new_new", "fc7-newb_new", "fc8-shapes_new"]:
    transfer_params(
        net_2fcs, layer,
        net_ensemble, '2fcs-{}'.format(layer)
    )

In [80]:
# 3FCS
for layer in ["fc6-new", "fc7-new", "fc7-newb", "fc8-shapes"]:
    transfer_params(
        net_3fcs, layer,
        net_ensemble, '3fcs-{}'.format(layer)
    )

In [81]:
net_ensemble.save('dump_model_ensemble_3_2_1.caffemodel')

### Ensemble 3-3-3-3

In [69]:
weights = np.array([ 0.2357, 0.2325, 0.2214 ])

In [70]:
weights / weights.sum()

array([ 0.34179234,  0.33715197,  0.32105568])

In [82]:
1 - 0.342 - 0.337

0.3209999999999999

In [84]:
model = 'nets/tracker.prototxt';
model_ensemble = 'nets/tracker_3_3_3.prototxt';
weights_1 = 'dump_model_2_iter_50000.caffemodel';
weights_2 = 'dump_model_1_iter_50000.caffemodel';
weights_3 = 'dump_model_4_iter_50000.caffemodel';

caffe.set_mode_gpu();
caffe.set_device(0);

In [85]:
net1 = caffe.Net(model, weights_1, caffe.TEST)
net2 = caffe.Net(model, weights_2, caffe.TEST)
net3 = caffe.Net(model, weights_3, caffe.TEST)
net_ensemble = caffe.Net(model_ensemble, weights_1, caffe.TEST)

In [87]:
net_ensemble.params

OrderedDict([('conv1', <caffe._caffe.BlobVec at 0x7fd52d2e5280>),
             ('conv2', <caffe._caffe.BlobVec at 0x7fd52d2ddf30>),
             ('conv3', <caffe._caffe.BlobVec at 0x7fd52d2d1830>),
             ('conv4', <caffe._caffe.BlobVec at 0x7fd52d2d1590>),
             ('conv5', <caffe._caffe.BlobVec at 0x7fd52d2d13d0>),
             ('conv1_p', <caffe._caffe.BlobVec at 0x7fd52d2d1440>),
             ('conv2_p', <caffe._caffe.BlobVec at 0x7fd52d2d14b0>),
             ('conv3_p', <caffe._caffe.BlobVec at 0x7fd52d2d1520>),
             ('conv4_p', <caffe._caffe.BlobVec at 0x7fd52d2d1670>),
             ('conv5_p', <caffe._caffe.BlobVec at 0x7fd52d2d16e0>),
             ('ens1-fc6-new', <caffe._caffe.BlobVec at 0x7fd52d2d1750>),
             ('ens1-fc7-new', <caffe._caffe.BlobVec at 0x7fd52d2d17c0>),
             ('ens1-fc7-newb', <caffe._caffe.BlobVec at 0x7fd52d2d1600>),
             ('ens1-fc8-shapes', <caffe._caffe.BlobVec at 0x7fd52d2d18a0>),
             ('ens2-fc6-new', <caf

In [88]:
for layer in ['fc6-new', 'fc7-new', 'fc7-newb', 'fc8-shapes']:
    transfer_params(
        net1, layer,
        net_ensemble, 'ens1-{}'.format(layer)
    )

In [89]:
for layer in ['fc6-new', 'fc7-new', 'fc7-newb', 'fc8-shapes']:
    transfer_params(
        net2, layer,
        net_ensemble, 'ens2-{}'.format(layer)
    )

In [90]:
for layer in ['fc6-new', 'fc7-new', 'fc7-newb', 'fc8-shapes']:
    transfer_params(
        net3, layer,
        net_ensemble, 'ens3-{}'.format(layer)
    )

In [91]:
net_ensemble.save('dump_model_ensemble_3_3_3.caffemodel')